In [0]:
def apply_incremental_load(df, table_name, partition_column):

    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

    if spark.catalog.tableExists(table_name):
        column_names = df.columns
        column_names.remove(partition_column)
        column_names.append(partition_column)
        df_new = df.select(column_names)
        df_new.write.mode("overwrite").insertInto(table_name)
    else:
        df.write.mode("overwrite").partitionBy(partition_column).format("parquet").saveAsTable(table_name)

In [0]:
from delta.tables import DeltaTable

def merge_delta_data(input_df, target_db_name, target_table_name, target_folder_path, merge_condition, partition_column):
    spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning.enabled", "True")

    if spark.catalog.tableExists(f"{target_db_name}.{target_table_name}"):
        delta_table = DeltaTable.forPath(spark, f"{target_folder_path}/{target_table_name}")

        delta_table.alias("tgt").merge(
            input_df.alias("src"),
            merge_condition) \
            .whenNotMatchedInsertAll() \
            .whenMatchedUpdateAll() \
            .execute()
    else:
        input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{target_db_name}.{target_table_name}")